This Transform notebook takes the three concatenated VMT files from csv and transforms them into the final VMT dataset that will be used for modeling. 

In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np
from matplotlib import pyplot as plt

2000-2008 reformat

In [4]:
df1 = pd.read_csv('Concat_2000_2008_vmt.csv')
df1 = df1.drop(columns=['AVMT (1,000)'])
df1.head()

,YEAR,STATE_CODE,COUNTY_CODE,F_SYSTEM,VMT_TOTAL
0,2000,1,1,1.0,182238280.8
1,2000,1,1,2.0,29225575.8
2,2000,1,1,14.0,62985415.8
3,2000,1,3,1.0,612965497.9
4,2000,1,3,2.0,469022778.0


In [5]:
df1 = df1.groupby(by=['YEAR', 'STATE_CODE', 'COUNTY_CODE']).sum()
df1.reset_index(inplace= True)
df1 = df1.drop(columns = ['F_SYSTEM'])

In [6]:
df1.head()

,YEAR,STATE_CODE,COUNTY_CODE,VMT_TOTAL
0,2000,1,1,2.744493e+08
1,2000,1,3,1.196020e+09
2,2000,1,5,1.276073e+08
3,2000,1,7,9.060923e+07
4,2000,1,9,2.181014e+08


2016-2020 reformat

In [7]:
df3 = pd.read_csv('County_VMT_2016-2020.csv').drop(columns = ['Unnamed: 6','Unnamed: 7','Total_daily_VMT'])
df3.columns = ['County_FIPS', 'STATE_CODE', 'COUNTY_CODE', 'YEAR', 'VMT_TOTAL']

In [8]:
df3.head()

,County_FIPS,STATE_CODE,COUNTY_CODE,YEAR,VMT_TOTAL
0,1001.0,1,1.0,2016,3.497314e+08
1,1003.0,1,3.0,2016,1.578541e+09
2,1005.0,1,5.0,2016,1.365250e+08
3,1007.0,1,7.0,2016,1.152019e+08
4,1009.0,1,9.0,2016,2.951796e+08


2011-2015 reformat

The challenge with 2011-2015 is that they are significantly higher values than 2000 - 2008 because they include local roads / non-federal roads, however, unlike 2016-2020 that also has this, they do not have VMT broken out by road type. Thus, we have to find a ratio to be able to get a better estimate of the actual data. 

In [9]:
df2 = pd.read_csv('Concat_2011_2015_vmt.csv')
df2 = df2.rename(columns = {'Unnamed: 5':'YEAR', 'COUNTY':'COUNTY_CODE', 'ST_code':'STATE_CODE'}).drop(columns = ['ST_name'])

In [10]:
df2.head()

,STATE_CODE,COUNTY_CODE,DVMT,Total_VMT,YEAR
0,1,1.0,"1,414,542",516307698.6,2011
1,1,3.0,"5,626,828",2053792271,2011
2,1,5.0,"792,480",289255302.2,2011
3,1,7.0,"503,159",183653206.6,2011
4,1,9.0,"1,289,855",470796932.7,2011


We apply the percentage of federal road vmt to all road vmt from 2016-2020 and multiply that percentage by vmt for 2011-2015

In [11]:
filtered_vmt = pd.read_csv('filtered_percent_imputation.csv').drop(columns = ['Filtered_total', 'Total', 'County_FIPS'])
filtered_vmt= filtered_vmt.pivot(index = ['ST','CO'], columns = ['Year'], values = ['Percentage'])


In [12]:
filtered_vmt

Percentage                                                    
Year            2016         2017         2018         2019         2020
ST CO                                                                   
1  1.0   0.627103714   0.63962731  0.650776292  0.646823718  0.641023986
   3.0   0.576228118  0.583518584   0.57778903  0.573308431  0.553403036
   5.0   0.514961521  0.508783553  0.510189119  0.511138384  0.509885318
   7.0   0.545611277  0.541999682  0.534537206  0.521493852  0.529833567
   9.0   0.571839329  0.571103807  0.557852067  0.636029638  0.634135508
...              ...          ...          ...          ...          ...
56 37.0  0.801421514  0.803739973  0.813508736  0.820173941  0.821713528
   39.0  0.538473228  0.535927983  0.514259304  0.440668343  0.511638404
   41.0  0.714128516  0.736421588  0.748027264  0.738069986  0.755974593
   43.0  0.712654189  0.707781384  0.707995053  0.716155505   0.71507872
   45.0  0.697711178  0.726639525  0.747572771  0.744342309  0.749046837

[3164 rows x 5 columns]

In [13]:
filtered_vmt.reset_index()


ST    CO   Percentage                                         \
Year                   2016         2017         2018         2019   
0      1   1.0  0.627103714   0.63962731  0.650776292  0.646823718   
1      1   3.0  0.576228118  0.583518584   0.57778903  0.573308431   
2      1   5.0  0.514961521  0.508783553  0.510189119  0.511138384   
3      1   7.0  0.545611277  0.541999682  0.534537206  0.521493852   
4      1   9.0  0.571839329  0.571103807  0.557852067  0.636029638   
...   ..   ...          ...          ...          ...          ...   
3159  56  37.0  0.801421514  0.803739973  0.813508736  0.820173941   
3160  56  39.0  0.538473228  0.535927983  0.514259304  0.440668343   
3161  56  41.0  0.714128516  0.736421588  0.748027264  0.738069986   
3162  56  43.0  0.712654189  0.707781384  0.707995053  0.716155505   
3163  56  45.0  0.697711178  0.726639525  0.747572771  0.744342309   

                   
Year         2020  
0     0.641023986  
1     0.553403036  
2     0.509885318  
3     0.529833567  
4     0.634135508  
...           ...  
3159  0.821713528  
3160  0.511638404  
3161  0.755974593  
3162   0.71507872  
3163  0.749046837  

[3164 rows x 7 columns]

In [14]:
#filtered_vmt.to_csv('percentages.csv')

Had some trouble here messing with the multiindex column headers of the filtered VMT data. I save the filtered_vmt dataframe to csv, and then open it in excel, fix the columns to be a single row, and then re-read the csv in.

In [15]:
percentages = pd.read_csv('percentages.csv')
percentages.head()

,ST,CO,2016,2017,2018,2019,2020,average
0,1,1.0,0.627104,0.63962731,0.650776,0.646823718,0.641024,0.641071004
1,1,3.0,0.576228,0.583518584,0.577789,0.573308431,0.553403,0.57284944
2,1,5.0,0.514962,0.508783553,0.510189,0.511138384,0.509885,0.510991579
3,1,7.0,0.545611,0.541999682,0.534537,0.521493852,0.529834,0.534695117
4,1,9.0,0.571839,0.571103807,0.557852,0.636029638,0.634136,0.59419207


In [16]:
average_percent = percentages[['ST', 'CO', 'average']].rename(columns = {'ST': 'STATE_CODE', 'CO':'COUNTY_CODE'})
average_percent

,STATE_CODE,COUNTY_CODE,average
0,1,1.0,0.641071004
1,1,3.0,0.57284944
2,1,5.0,0.510991579
3,1,7.0,0.534695117
4,1,9.0,0.59419207
...,...,...,...
3159,56,37.0,0.812111538
3160,56,39.0,0.508193452
3161,56,41.0,0.738524389
3162,56,43.0,0.71193297


In [17]:
df2_final = df2.merge(average_percent, left_on = ['STATE_CODE', 'COUNTY_CODE'], right_on =['STATE_CODE', 'COUNTY_CODE'])
df2_final['average'] = pd.to_numeric(df2_final['average'], downcast='float', errors='coerce')
df2_final['Total_VMT'] = pd.to_numeric(df2_final['Total_VMT'], downcast='float', errors='coerce')

df2_final['Total_VMT_Filtered'] = df2_final['Total_VMT'] * df2_final['average']

In [18]:
df2 = df2_final[['STATE_CODE','COUNTY_CODE', 'YEAR', 'Total_VMT_Filtered']].rename(columns = {'Total_VMT_Filtered':'Total_VMT'})

Concat

In [19]:
df_concat = pd.concat([df1[['YEAR','STATE_CODE','COUNTY_CODE','VMT_TOTAL']],
                      df2.rename(columns={'Total_VMT':'VMT_TOTAL', 'COUNTY': 'COUNTY_CODE', 'ST_code':'STATE_CODE'})[['YEAR','STATE_CODE','COUNTY_CODE','VMT_TOTAL']],
                      df3[['YEAR','STATE_CODE','COUNTY_CODE','VMT_TOTAL']]
                      ])

#df_concat.dtypes

In [20]:
#if there is daily vmt, we multiply by 365, or 366 on leap year.
df_concat['Daily_VMT'] = np.where(df_concat['YEAR']%4 == 0, df_concat['VMT_TOTAL']/366, df_concat['VMT_TOTAL']/365)
df_concat.head()

,YEAR,STATE_CODE,COUNTY_CODE,VMT_TOTAL,Daily_VMT
0,2000,1,1.0,2.744493e+08,7.498614e+05
1,2000,1,3.0,1.196020e+09,3.267816e+06
2,2000,1,5.0,1.276073e+08,3.486537e+05
3,2000,1,7.0,9.060923e+07,2.475662e+05
4,2000,1,9.0,2.181014e+08,5.959055e+05


In [21]:
df_yearly = df_concat[['YEAR','STATE_CODE','COUNTY_CODE','VMT_TOTAL']]
df_yearly = df_yearly.pivot(index = ['STATE_CODE','COUNTY_CODE'], columns = 'YEAR', values = 'VMT_TOTAL')
df_yearly.head()

YEAR                            2000          2001          2002  \
STATE_CODE COUNTY_CODE                                             
1          1.0          2.744493e+08  2.943835e+08  3.027907e+08   
           3.0          1.196020e+09  1.167018e+09  1.231052e+09   
           5.0          1.276073e+08  1.261550e+08  1.334990e+08   
           7.0          9.060923e+07  8.926013e+07  8.489820e+07   
           9.0          2.181014e+08  2.166068e+08  2.215053e+08   

YEAR                            2003          2004          2005  \
STATE_CODE COUNTY_CODE                                             
1          1.0          3.082386e+08  3.216756e+08  3.229936e+08   
           3.0          1.258745e+09  1.331049e+09  1.340592e+09   
           5.0          1.394845e+08  1.337402e+08  1.371450e+08   
           7.0          8.450337e+07  9.002441e+07  1.077608e+08   
           9.0          2.187804e+08  2.203252e+08  2.195224e+08   

YEAR                            2006          2007          2008  \
STATE_CODE COUNTY_CODE                                             
1          1.0          3.350919e+08  3.369326e+08  3.082316e+08   
           3.0          1.418405e+09  1.405449e+09  1.345076e+09   
           5.0          1.408645e+08  1.455554e+08  1.356890e+08   
           7.0          1.050904e+08  9.894335e+07  9.530857e+07   
           9.0          2.200891e+08  2.198925e+08  2.126830e+08   

YEAR                            2011          2012          2013  \
STATE_CODE COUNTY_CODE                                             
1          1.0          3.309899e+08  3.360271e+08  3.320919e+08   
           3.0          1.176514e+09  1.227559e+09  1.333338e+09   
           5.0          1.478070e+08  1.466344e+08  1.411944e+08   
           7.0          9.819846e+07  1.027386e+08  1.039564e+08   
           9.0          2.797438e+08  2.794617e+08  2.808611e+08   

YEAR                            2014          2015          2016  \
STATE_CODE COUNTY_CODE                                             
1          1.0          3.371580e+08  3.384281e+08  3.497314e+08   
           3.0          1.418520e+09  1.496820e+09  1.578541e+09   
           5.0          1.361059e+08  1.320808e+08  1.365250e+08   
           7.0          1.024895e+08  1.080322e+08  1.152019e+08   
           9.0          2.831056e+08  3.022390e+08  2.951796e+08   

YEAR                            2017          2018          2019          2020  
STATE_CODE COUNTY_CODE                                                          
1          1.0          3.688914e+08  3.837076e+08  3.857982e+08  3.725326e+08  
           3.0          1.644976e+09  1.629884e+09  1.662900e+09  1.555627e+09  
           5.0          1.352483e+08  1.367523e+08  1.358917e+08  1.318903e+08  
           7.0          1.139288e+08  1.100951e+08  1.105581e+08  1.093106e+08  
           9.0          2.912142e+08  2.836509e+08  3.290431e+08  3.075544e+08

In [22]:
np.array(df_yearly.iloc[1])

array([1.19602048e+09, 1.16701844e+09, 1.23105222e+09, 1.25874506e+09,
       1.33104900e+09, 1.34059177e+09, 1.41840452e+09, 1.40544882e+09,
       1.34507562e+09, 1.17651379e+09, 1.22755853e+09, 1.33333824e+09,
       1.41852019e+09, 1.49682022e+09, 1.57854137e+09, 1.64497576e+09,
       1.62988447e+09, 1.66289956e+09, 1.55562654e+09])

In [48]:
#df_yearly.to_csv('COUNTY_YEARLY_DATA.csv')

Impute the middle years of the data

In [24]:
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [23]:
df = pd.read_csv('COUNTY_YEARLY_DATA.csv')
df

,STATE_CODE,COUNTY_CODE,2000,2001,2002,2003,2004,2005,2006,2007,2008,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,1.0,2.744493e+08,2.943835e+08,3.027907e+08,3.082386e+08,3.216756e+08,3.229936e+08,3.350919e+08,3.369326e+08,3.082316e+08,3.309899e+08,3.360271e+08,3.320919e+08,3.371580e+08,3.384281e+08,3.497314e+08,3.688914e+08,3.837076e+08,3.857982e+08,3.725326e+08
1,1,3.0,1.196020e+09,1.167018e+09,1.231052e+09,1.258745e+09,1.331049e+09,1.340592e+09,1.418405e+09,1.405449e+09,1.345076e+09,1.176514e+09,1.227559e+09,1.333338e+09,1.418520e+09,1.496820e+09,1.578541e+09,1.644976e+09,1.629884e+09,1.662900e+09,1.555627e+09
2,1,5.0,1.276073e+08,1.261550e+08,1.334990e+08,1.394845e+08,1.337402e+08,1.371450e+08,1.408645e+08,1.455554e+08,1.356890e+08,1.478070e+08,1.466344e+08,1.411944e+08,1.361059e+08,1.320808e+08,1.365250e+08,1.352483e+08,1.367523e+08,1.358917e+08,1.318903e+08
3,1,7.0,9.060923e+07,8.926013e+07,8.489820e+07,8.450337e+07,9.002441e+07,1.077608e+08,1.050904e+08,9.894335e+07,9.530857e+07,9.819846e+07,1.027386e+08,1.039564e+08,1.024895e+08,1.080322e+08,1.152019e+08,1.139288e+08,1.100951e+08,1.105581e+08,1.093106e+08
4,1,9.0,2.181014e+08,2.166068e+08,2.215053e+08,2.187804e+08,2.203252e+08,2.195224e+08,2.200891e+08,2.198925e+08,2.126830e+08,2.797438e+08,2.794617e+08,2.808611e+08,2.831056e+08,3.022390e+08,2.951796e+08,2.912142e+08,2.836509e+08,3.290431e+08,3.075544e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,72,3.0,6.314885e+08,6.451371e+08,6.564721e+08,8.824500e+08,8.789018e+08,8.841088e+08,8.779714e+08,8.636822e+08,8.394342e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3176,72,4.0,6.013523e+08,6.120455e+08,6.000972e+08,7.272984e+08,7.571927e+08,7.607727e+08,7.692118e+08,7.428546e+08,7.379312e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3177,72,5.0,7.927161e+08,8.055066e+08,8.494390e+08,8.977984e+08,9.486458e+08,9.501677e+08,9.426519e+08,8.596167e+08,8.361125e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3178,72,6.0,5.689777e+08,5.765905e+08,6.010404e+08,5.994609e+08,6.159061e+08,6.107699e+08,5.984812e+08,6.002262e+08,6.167818e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#remove counties that don't have at least 10 non-zero data points
keep_list = []

for i in range(df.shape[0]):
    df_stg = df.drop(columns = ['STATE_CODE', 'COUNTY_CODE'])
    df_stg = df_stg.iloc[i].dropna()
    if len(df_stg[df_stg!=0].dropna()) >= 10:
        keep_list.append(1)
    else:
        keep_list.append(0)
    
df['rows_to_keep'] = keep_list


In [25]:
print('total counties kept equals', df['rows_to_keep'].sum())
print('percent counties kept equals', df['rows_to_keep'].mean())



total counties kept equals 2898
percent counties kept equals 0.9113207547169812


In [26]:
df = df[df['rows_to_keep'] == 1].drop(columns = ['rows_to_keep'])
df

,STATE_CODE,COUNTY_CODE,2000,2001,2002,2003,2004,2005,2006,2007,2008,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,1.0,2.744493e+08,2.943835e+08,3.027907e+08,3.082386e+08,3.216756e+08,3.229936e+08,3.350919e+08,3.369326e+08,3.082316e+08,3.309899e+08,3.360271e+08,3.320919e+08,3.371580e+08,3.384281e+08,3.497314e+08,3.688914e+08,3.837076e+08,3.857982e+08,3.725326e+08
1,1,3.0,1.196020e+09,1.167018e+09,1.231052e+09,1.258745e+09,1.331049e+09,1.340592e+09,1.418405e+09,1.405449e+09,1.345076e+09,1.176514e+09,1.227559e+09,1.333338e+09,1.418520e+09,1.496820e+09,1.578541e+09,1.644976e+09,1.629884e+09,1.662900e+09,1.555627e+09
2,1,5.0,1.276073e+08,1.261550e+08,1.334990e+08,1.394845e+08,1.337402e+08,1.371450e+08,1.408645e+08,1.455554e+08,1.356890e+08,1.478070e+08,1.466344e+08,1.411944e+08,1.361059e+08,1.320808e+08,1.365250e+08,1.352483e+08,1.367523e+08,1.358917e+08,1.318903e+08
3,1,7.0,9.060923e+07,8.926013e+07,8.489820e+07,8.450337e+07,9.002441e+07,1.077608e+08,1.050904e+08,9.894335e+07,9.530857e+07,9.819846e+07,1.027386e+08,1.039564e+08,1.024895e+08,1.080322e+08,1.152019e+08,1.139288e+08,1.100951e+08,1.105581e+08,1.093106e+08
4,1,9.0,2.181014e+08,2.166068e+08,2.215053e+08,2.187804e+08,2.203252e+08,2.195224e+08,2.200891e+08,2.198925e+08,2.126830e+08,2.797438e+08,2.794617e+08,2.808611e+08,2.831056e+08,3.022390e+08,2.951796e+08,2.912142e+08,2.836509e+08,3.290431e+08,3.075544e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3167,56,37.0,7.539353e+08,7.573229e+08,7.732578e+08,8.292675e+08,7.835613e+08,7.933938e+08,8.987296e+08,8.643804e+08,8.354893e+08,8.523962e+08,8.143411e+08,8.034601e+08,8.094254e+08,8.172584e+08,7.589328e+08,8.063954e+08,8.350411e+08,9.023661e+08,8.070806e+08
3168,56,39.0,1.459938e+08,1.537269e+08,1.572646e+08,1.622182e+08,1.581519e+08,1.529649e+08,1.583781e+08,1.516270e+08,1.531311e+08,1.568753e+08,1.541297e+08,1.524393e+08,1.570518e+08,1.655318e+08,1.710872e+08,1.773861e+08,1.826048e+08,1.842755e+08,1.738602e+08
3169,56,41.0,2.567823e+08,2.536745e+08,2.696866e+08,2.658505e+08,2.690057e+08,2.764732e+08,2.946376e+08,2.917407e+08,2.714784e+08,2.610719e+08,2.534195e+08,2.522354e+08,2.586523e+08,2.602600e+08,2.548959e+08,2.645529e+08,2.823919e+08,2.915467e+08,2.682344e+08
3170,56,43.0,5.686812e+07,6.186890e+07,6.362864e+07,6.382171e+07,6.010312e+07,5.695633e+07,5.374279e+07,5.169416e+07,5.042345e+07,5.076674e+07,5.188140e+07,5.121330e+07,5.034845e+07,5.120782e+07,5.050928e+07,4.560024e+07,4.809015e+07,5.165241e+07,4.965127e+07


In [ ]:
#imputation of 2009 and 2010 variables


In [30]:
'''#imputing the data here so that all data now has 2009 and 2010 data if meet reasonable parameters
values = []

regression_data = df.drop(columns = ['STATE_CODE', 'COUNTY_CODE'])
years = np.array(list(range(2000,2009)) + list(range(2011,2021))).reshape(-1,1)

for i in tqdm(range(df.shape[0])):
    a = regression_data.iloc[i].dropna()
    a = a[a>0]
    y = np.array(a)
    X = np.array(a.index).astype(int).reshape(-1,1)
    if len(X) < 8 or max(X) < 2010:
        values.append(np.array([[np.nan],[np.nan]]))
    else:
        reg = LinearRegression().fit(X, y)
        values.append(reg.predict(np.array([[2009], [2010]])))
    
imputed_values = pd.DataFrame(values, columns = ['2009', '2010'])
df['2009'] = imputed_values['2009']
df['2010'] = imputed_values['2010']


#undo comment to get imputations


'''

100%|██████████| 2898/2898 [00:01<00:00, 2569.61it/s]


In [26]:
#df[['STATE_CODE', 'COUNTY_CODE', '2000','2001','2002','2003','2004','2005','2006','2007','2008','2009',
#   '2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']].to_csv('Imputed_County_VMT_Final.csv')


IndentationError: unexpected indent (849533533.py, line 2)

In [53]:
df['2009'] = np.nan
df['2010'] = np.nan

df[['STATE_CODE', 'COUNTY_CODE', '2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010',
    '2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']].to_csv('Non-Imputed_County_VMT_Final.csv')

In [57]:
df

,STATE_CODE,COUNTY_CODE,2000,2001,2002,2003,2004,2005,2006,2007,2008,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2009,2010
0,1,1.0,2.744493e+08,2.943835e+08,3.027907e+08,3.082386e+08,3.216756e+08,3.229936e+08,3.350919e+08,3.369326e+08,3.082316e+08,3.309899e+08,3.360271e+08,3.320919e+08,3.371580e+08,3.384281e+08,3.497314e+08,3.688914e+08,3.837076e+08,3.857982e+08,3.725326e+08,NaN,NaN
1,1,3.0,1.196020e+09,1.167018e+09,1.231052e+09,1.258745e+09,1.331049e+09,1.340592e+09,1.418405e+09,1.405449e+09,1.345076e+09,1.176514e+09,1.227559e+09,1.333338e+09,1.418520e+09,1.496820e+09,1.578541e+09,1.644976e+09,1.629884e+09,1.662900e+09,1.555627e+09,NaN,NaN
2,1,5.0,1.276073e+08,1.261550e+08,1.334990e+08,1.394845e+08,1.337402e+08,1.371450e+08,1.408645e+08,1.455554e+08,1.356890e+08,1.478070e+08,1.466344e+08,1.411944e+08,1.361059e+08,1.320808e+08,1.365250e+08,1.352483e+08,1.367523e+08,1.358917e+08,1.318903e+08,NaN,NaN
3,1,7.0,9.060923e+07,8.926013e+07,8.489820e+07,8.450337e+07,9.002441e+07,1.077608e+08,1.050904e+08,9.894335e+07,9.530857e+07,9.819846e+07,1.027386e+08,1.039564e+08,1.024895e+08,1.080322e+08,1.152019e+08,1.139288e+08,1.100951e+08,1.105581e+08,1.093106e+08,NaN,NaN
4,1,9.0,2.181014e+08,2.166068e+08,2.215053e+08,2.187804e+08,2.203252e+08,2.195224e+08,2.200891e+08,2.198925e+08,2.126830e+08,2.797438e+08,2.794617e+08,2.808611e+08,2.831056e+08,3.022390e+08,2.951796e+08,2.912142e+08,2.836509e+08,3.290431e+08,3.075544e+08,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3167,56,37.0,7.539353e+08,7.573229e+08,7.732578e+08,8.292675e+08,7.835613e+08,7.933938e+08,8.987296e+08,8.643804e+08,8.354893e+08,8.523962e+08,8.143411e+08,8.034601e+08,8.094254e+08,8.172584e+08,7.589328e+08,8.063954e+08,8.350411e+08,9.023661e+08,8.070806e+08,NaN,NaN
3168,56,39.0,1.459938e+08,1.537269e+08,1.572646e+08,1.622182e+08,1.581519e+08,1.529649e+08,1.583781e+08,1.516270e+08,1.531311e+08,1.568753e+08,1.541297e+08,1.524393e+08,1.570518e+08,1.655318e+08,1.710872e+08,1.773861e+08,1.826048e+08,1.842755e+08,1.738602e+08,NaN,NaN
3169,56,41.0,2.567823e+08,2.536745e+08,2.696866e+08,2.658505e+08,2.690057e+08,2.764732e+08,2.946376e+08,2.917407e+08,2.714784e+08,2.610719e+08,2.534195e+08,2.522354e+08,2.586523e+08,2.602600e+08,2.548959e+08,2.645529e+08,2.823919e+08,2.915467e+08,2.682344e+08,NaN,NaN
3170,56,43.0,5.686812e+07,6.186890e+07,6.362864e+07,6.382171e+07,6.010312e+07,5.695633e+07,5.374279e+07,5.169416e+07,5.042345e+07,5.076674e+07,5.188140e+07,5.121330e+07,5.034845e+07,5.120782e+07,5.050928e+07,4.560024e+07,4.809015e+07,5.165241e+07,4.965127e+07,NaN,NaN
